In [2]:
import numpy as np
import matplotlib.pyplot as plt

##### The current training data, is aixs_magnitude_train_data, train_mean.npy, train_std.npy
##### The problem is they are incorrectly normalized. The axis_magnitude from axis_angle conversion was done post normalization
##### Normalization should have been the last step in the process

In [3]:
train_data_loc = 'data_more_processing/axis_magnitude_train_data.pkl'
train_data = np.load(train_data_loc, allow_pickle=True)

In [4]:
type(train_data), len(train_data), type(train_data[0])

(list, 994, tuple)

In [5]:
train_data[0][0].shape

(300, 242)

##### The flow of data in the above case has been.
##### Raw-SMPLX -> Crop and Pad -> Normalize (Mean and Std) -> Axis-Angle to Axis-Magnitude
##### (L, 187) -> (T, 187) -> (T, 187) -> (T, 242)

##### But instead it should have been 
##### Raw-SMPLX -> Crop and Pad -> Axis-Angle to Axis-Magnitude -> Normalize (-1 to 1)
##### (L, 187) -> (T, 187) -> (T, 242) -> (T, 242)

##### Because of this error, now you will need to change the training code to make sure, 
##### All the reverse steps make sense. Go through the training code

In [6]:
train_data_smplx = np.load('train_data_smplx.pkl', allow_pickle=True)
test_data_smplx = np.load('test_data_smplx.pkl', allow_pickle=True)

In [7]:
train_data_smplx[0][0].shape

(300, 187)

In [8]:
# The above is Crop and Pad data, 3 + 3 + 165 + 16 = 187
# (transl, global_orient, poses, betas)

In [9]:
def convert_to_axis_magnitude(data):
    '''data [T, 187]'''
    poses = data[:, 6:171].reshape(-1, 55, 3)
    magnitude = np.linalg.norm(poses, axis=2, keepdims=True)
    unit_axis = poses / (magnitude + 1e-10)
    axis_magnitude = np.concatenate([unit_axis, magnitude], axis=2).reshape(poses.shape[0], -1)
    new_data = np.concatenate([data[:, :6], axis_magnitude, data[:, 171:]], axis=1)
    return new_data

In [10]:
def convert_to_axis_angle(data):
    '''data [T, 242]'''
    axis_magnitude = data[:, 6:226].reshape(-1, 55, 4)
    axis = axis_magnitude[:, :, :3]
    magnitude = axis_magnitude[:, :, 3:]
    poses = axis * (magnitude+1e-10)
    poses = poses.reshape(axis_magnitude.shape[0], -1)
    new_data = torch.cat([data[:, :6], poses, data[:, 226:]], axis=1)
    return new_data

In [11]:
axis_magnitude_train_data = [(convert_to_axis_magnitude(T), convert_to_axis_magnitude(D)) for T, D in train_data_smplx]

In [12]:
axis_magnitude_test_data = [(convert_to_axis_magnitude(T), convert_to_axis_magnitude(D)) for T, D in test_data_smplx]

In [13]:
all_values = np.concatenate([np.concatenate([L.flatten(), F.flatten()]) for L, F in axis_magnitude_train_data])
global_min = all_values.min()
global_max = all_values.max()

In [14]:
global_min, global_max

(-5.049882127052023, 6.446737822522641)

In [15]:
def normalize_global(array, min_val, max_val):
    if max_val == min_val:
        return np.zeros_like(array)
    return 2 * (array - min_val) / (max_val - min_val) - 1

In [22]:
def denormalize_global(array, min_val, max_val):
    if max_val == min_val:
        return np.zeros_like(array)
    return (array + 1) * (max_val - min_val) / 2 + min_val

In [24]:
(global_min, global_max) = (-5.049882127052023, 6.446737822522641) 

In [16]:
normalized_axis_magnitude_train_data = [
    (normalize_global(L, global_min, global_max), normalize_global(F, global_min, global_max))
    for L, F in axis_magnitude_train_data
]

In [17]:
normalized_axis_magnitude_test_data = [
    (normalize_global(L, global_min, global_max), normalize_global(F, global_min, global_max))
    for L, F in axis_magnitude_test_data
]

In [21]:
# Save the normalized data
np.save('normalized_axis_magnitude_train_data_new.npy', normalized_axis_magnitude_train_data)
np.save('normalized_axis_magnitude_test_data_new.npy', normalized_axis_magnitude_test_data)

In [28]:
# Now just for once verify if a sample data-point from this after making reverse changes makes sense when plotted